In [3]:
# Screening HIV-1 sequences worldwide

import RNA

def DNA_reverse_complement_RNA(DNA):
    complement = {'A': 'U', 'C': 'G', 'G': 'C', 'T': 'A'}
    return ''.join(complement.get(base, base) for base in reversed(DNA))

# Only one base of permutation is tolerated and it should happen at the first two bases counting from the 3' end 
def single_base_permutation_3end_preserve_forward(DNA):
    permutation_set = []
    for i in range (1, len(DNA)-1):
        permutation = DNA[:i-1] + 'A' + DNA[i:] 
        permutation_set.append(permutation)
        permutation = DNA[:i-1] + 'C' + DNA[i:] 
        permutation_set.append(permutation)
        permutation = DNA[:i-1] + 'G' + DNA[i:] 
        permutation_set.append(permutation)
        permutation = DNA[:i-1] + 'T' + DNA[i:] 
        permutation_set.append(permutation)
    return permutation_set

def single_base_permutation_3end_preserve_reverse(DNA):
    permutation_set = []
    for i in range (4, len(DNA)+1):
        permutation = DNA[:i-1] + 'A' + DNA[i:] 
        permutation_set.append(permutation)
        permutation = DNA[:i-1] + 'C' + DNA[i:] 
        permutation_set.append(permutation)
        permutation = DNA[:i-1] + 'G' + DNA[i:] 
        permutation_set.append(permutation)
        permutation = DNA[:i-1] + 'T' + DNA[i:] 
        permutation_set.append(permutation)
    return permutation_set

# Only one base of insertion is tolerated, the insertion site should not be at the first two bases counting the 3' end or their closest neighbors  
def single_base_insertion_3end_preserve_forward(DNA):
    insertion_set = []
    for i in range (1, len(DNA)-1):
        insertion = DNA[:i-1] + 'A' + DNA[i-1:]
        insertion_set.append(insertion)
        insertion = DNA[:i-1] + 'C' + DNA[i-1:]
        insertion_set.append(insertion)
        insertion = DNA[:i-1] + 'G' + DNA[i-1:]
        insertion_set.append(insertion)
        insertion = DNA[:i-1] + 'T' + DNA[i-1:]
        insertion_set.append(insertion)
    return insertion_set
        
def single_base_insertion_3end_preserve_reverse(DNA):
    insertion_set = []
    for i in range (4, len(DNA)+1):
        insertion = DNA[:i-1] + 'A' + DNA[i-1:]
        insertion_set.append(insertion)
        insertion = DNA[:i-1] + 'C' + DNA[i-1:]
        insertion_set.append(insertion)
        insertion = DNA[:i-1] + 'G' + DNA[i-1:]
        insertion_set.append(insertion)
        insertion = DNA[:i-1] + 'T' + DNA[i-1:]
        insertion_set.append(insertion)
    return insertion_set

# Only one base of deletion is tolerated, the deleted base should not be the first two bases counting the 3' end or their closest neighbors  
def single_base_deletion_3end_preserve_forward(DNA):
    deletion_set = []
    for i in range (1, len(DNA)-2):
        deletion = DNA[:i-1] + DNA[i:]
    return deletion_set
    
def single_base_deletion_3end_preserve_reverse(DNA):
    deletion_set = []
    for i in range (4, len(DNA)):
        deletion = DNA[:i-1] + DNA[i:]
    return deletion_set
        
ref_amplicon = 'AATGAGGAAGCTGCAGAATGGGATAGAGTGCATCCAGTGCATGCAGGGCCTATTGCACCAGGCCAGATGAGAGAACCAAGGGG'    # Amplicon sequence from the reference HIV-1 genome (NC001802.1)
forward = 'AAGCAGCCATGCAAATGTTAAAAGAGACCATC'    # Forward RPA primer from the reference genome 
reverse = 'GAACCAAGGGGAAGTGACATAGCAGGAACTAC'    # Reverse RPA primer from the reference genome 
print('forward primer length: ' + str(len(forward)))
print('reverse primer length: ' + str(len(reverse)))

# Generating a primer set containing satisfying primers to be matched to sequencing records 
permutation_set_forward = single_base_permutation_3end_preserve_forward(forward)
permutation_set_reverse = single_base_permutation_3end_preserve_reverse(reverse)
insertion_set_forward = single_base_insertion_3end_preserve_forward(forward)
insertion_set_reverse = single_base_insertion_3end_preserve_reverse(reverse)
deletion_set_forward = single_base_deletion_3end_preserve_forward(forward)
deletion_set_reverse = single_base_deletion_3end_preserve_reverse(reverse)

# World wide HIV-1 sequencing records
file = open('sequences-HIV-1-worldwide.fasta', 'r')  

# North America HIV-1 sequencing records, remove the top and bottom semicolon to enable
'''
file = open('sequences-HIV-1-North America.fasta', 'r') 
'''

# FASTA file processing
lines = file.readlines()
whole_seq = []
for line in lines:
    if line[0] == '>':
        whole_seq.append('\n')
        whole_seq.append(line)
        continue
    else:
        line = line.strip()
        whole_seq.append(line)
    
whole_seq = ''.join(whole_seq)
whole_seq = whole_seq[1:]

whole_seq = whole_seq.splitlines()
total_seq_num = int(len(whole_seq)/2)    # Return total number of sequencing records 
print('Total sequence number in database: ' + str(total_seq_num))

# primer screen, the amplicon sequences with matched primers will be extracted for further processing
amplicon_set = []
primer_match_num = 0    # Initialize number of matched primer pairs 

for i in range (0, len(whole_seq)):
    if i%2 == 1:
        start = -1
        end = -1
        for j in range (0, len(whole_seq[i])-32+1):
            if whole_seq[i][j:j+32] in permutation_set_forward:
                start = j+32    # If yes, get the start base position
                break
        if start == -1:
            for j in range (0, len(whole_seq[i])-33+1):
                if whole_seq[i][j:j+33] in insertion_set_forward:
                    start = j+33    # If yes, get the start base position
                    break
            if start == -1:
                for j in range (0, len(whole_seq[i])-31+1):
                    if whole_seq[i][j:j+31] in deletion_set_forward:
                        start = j+31    # If yes, get the start base position
                        break

        for j in range (0, len(whole_seq[i])-32+1):
            if whole_seq[i][j:j+32] in permutation_set_reverse:
                end = j    # If yes, get the end base position
                break
        if start == -1:
            for j in range (0, len(whole_seq[i])-33+1):
                if whole_seq[i][j:j+33] in insertion_set_reverse:
                    end = j    # If yes, get the end base position
                    break
            if start == -1:
                for j in range (0, len(whole_seq[i])-31+1):
                    if whole_seq[i][j:j+31] in deletion_set_reverse:
                        end = j    # If yes, get the end base position
                        break

        if start != -1 and end != -1:
            primer_match_num += 1    # If both start and end base position are confirmed, which means both the forward and reverse primer are satisfying, then store this sequencing record and move on to CRISPR target screening
            insert = whole_seq[i][start:end+11]
            amplicon_set.append(insert)


print('Matched primer number: ' + str(primer_match_num))   # Return total number of sequencing records containing valid amplicons     
ratio_1 = primer_match_num/total_seq_num
print('primer match ratio: ' + str(ratio_1))    # Return the ratio of number of valid amplicons to the total number of sequencing records
  
print('----------------------------------------')    # Separator 
    
# CRISPR target mismatch screening
for s in range (0, len(ref_amplicon)-20+1):
    ref_target = ref_amplicon[s:s+20]    # Target length fixed to 20 nt, can be customized 

    target_match_num = 0    # Initialize number of matched targets  

    for i in range (0, len(amplicon_set)):
        error_flag = 1    # If error_flag keeps 1 after screening, the target did not pass and should be discarded 
        for k in range (0, len(amplicon_set[i])-20+1):
            if amplicon_set[i][k:k+20] == ref_target:
                error_flag = -1    # error_flag changes to -1 if the target meets the screening criteria 
                break
                        
        if error_flag == -1:
            target_match_num += 1

    print('Matched target number: ' + str(target_match_num))   # Return number of matched targets
    ratio_2 = target_match_num/primer_match_num    # Calculate the match ratio 
    print('target match ratio: ' + str(ratio_2))   # Return ratio of matched targets to the number of valid amplicons 
    print(ratio_2)

forward primer length: 32
reverse primer length: 32
Total sequence number in database: 1061945
Matched primer number: 43320
primer match ratio: 0.04079307308758928
----------------------------------------
Matched target number: 24263
target match ratio: 0.5600877192982456
0.5600877192982456
Matched target number: 24260
target match ratio: 0.5600184672206833
0.5600184672206833
Matched target number: 24238
target match ratio: 0.5595106186518929
0.5595106186518929
Matched target number: 24327
target match ratio: 0.5615650969529086
0.5615650969529086
Matched target number: 23274
target match ratio: 0.5372576177285319
0.5372576177285319
Matched target number: 23266
target match ratio: 0.5370729455216989
0.5370729455216989
Matched target number: 24783
target match ratio: 0.5720914127423823
0.5720914127423823
Matched target number: 21924
target match ratio: 0.5060941828254848
0.5060941828254848
Matched target number: 3933
target match ratio: 0.09078947368421053
0.09078947368421053
Matched tar